<a href="https://colab.research.google.com/github/stevansehn/python-amazon-bedrock-sentiment-analysis/blob/main/amazon_bedrock_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install boto3

In [2]:
from google.colab import drive
import os
import boto3

drive.mount('/content/drive')

credentials_path = '/content/drive/MyDrive/APIKeys/amazon-bedrock-user-credentials.txt'

with open(credentials_path) as f:
    content = f.read().strip()

lines = content.split('\n')

if len(lines) == 2:
    aws_access_key_id = lines[0].strip()
    aws_secret_access_key = lines[1].strip()
else:
    raise ValueError("Credentials file must contain exactly two lines: access key and secret key.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
os.environ['AWS_ACCESS_KEY_ID'] = aws_access_key_id
os.environ['AWS_SECRET_ACCESS_KEY'] = aws_secret_access_key

client = boto3.client("bedrock-runtime", region_name="us-east-1")

model_id = "amazon.titan-text-express-v1"

user_message = """
Meeting transcript:
Miguel: Hi Brant, I want to discuss the workstream  for our new product launch
Brant: Sure Miguel, is there anything in particular you want to discuss?
Miguel: Yes, I want to talk about how users enter into the product.
Brant: Ok, in that case let me add in Namita.
Namita: Hey everyone
Brant: Hi Namita, Miguel wants to discuss how users enter into the product.
Miguel: its too complicated and we should remove friction.  for example, why do I need to fill out additional forms?  I also find it difficult to find where to access the product when I first land on the landing page.
Brant: I would also add that I think there are too many steps.
Namita: Ok, I can work on the landing page to make the product more discoverable but brant can you work on the additonal forms?
Brant: Yes but I would need to work with James from another team as he needs to unblock the sign up workflow.  Miguel can you document any other concerns so that I can discuss with James only once?
Miguel: Sure.
From the meeting transcript above, analyze the sentiments of each character in the conversation.
"""

conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]

try:
    response = client.converse(
        modelId=model_id,
        messages=conversation,
        inferenceConfig={"maxTokens":4096,"stopSequences":["User:"],"temperature":0,"topP":1},
        additionalModelRequestFields={}
    )

    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)

except Exception as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")

Miguel: Miguel is frustrated with the complexity of the user onboarding process and wants to remove friction. He also finds it difficult to find the product when he first lands on the landing page.

Brant: Brant agrees that the user onboarding process is too complicated and suggests adding Namita to the conversation.

Namita: Namita agrees that the user onboarding process is too complicated and suggests making the product more discoverable on the landing page. She also suggests that Brant work on the additional forms, but Brant needs to work with James from another team to unblock the sign-up workflow.

Brant: Brant agrees that the user onboarding process is too complicated and suggests making the product more discoverable on the landing page. He also suggests that Miguel document any other concerns so that he can discuss with James only once.

Miguel: Miguel agrees that the user onboarding process is too complicated and suggests making the product more discoverable on the landing page